# Installation

In a conda environment with `enterprise` installed, one can easily install QuickCW by cloning the GitHub repo (https://github.com/bencebecsy/QuickCW/) and from the repo install it with pip as `pip install -e .`. More details can be found at https://github.com/bencebecsy/QuickCW/blob/main/docs/how_to_run_QuickCW.md 

# Running

In [1]:
#imports
import numpy as np
np.seterr(all='raise')
import matplotlib.pyplot as plt
#import corner

import pickle

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const

from enterprise_extensions import deterministic

#import glob
#import json

import QuickCW.QuickCW as QuickCW
from QuickCW.QuickMCMCUtils import ChainParams

Number of cores used for parallel running:  8


In [2]:
%%time
#Setting everything up for the sampler - this should run in a couple minutes
#and will require a good amount of RAM (10+ GB)

#make sure this points to the pickled pulsars you want to analyze
data_pkl = 'data/v1p1_de440_pint_bipm2019_unshifted_pdist.pkl'

with open(data_pkl, 'rb') as psr_pkl:
    psrs = pickle.load(psr_pkl)

print(len(psrs))

#number of iterations (increase to 100 million - 1 billion for actual analysis)
N = 5_000_000

n_int_block = 10_000 #number of iterations in a block (which has one shape update and the rest are projection updates)
save_every_n = 100_000 #number of iterations between saving intermediate results (needs to be intiger multiple of n_int_block)
N_blocks = np.int64(N//n_int_block) #number of blocks to do
fisher_eig_downsample = 2000 #multiplier for how much less to do more expensive updates to fisher eigendirections for red noise and common parameters compared to diagonal elements

n_status_update = 100 #number of status update printouts (N/n_status_update needs to be an intiger multiple of n_int_block)
n_block_status_update = np.int64(N_blocks//n_status_update) #number of bllocks between status updates

assert N_blocks%n_status_update ==0 #or we won't print status updates
assert N%save_every_n == 0 #or we won't save a complete block
assert N%n_int_block == 0 #or we won't execute the right number of blocks

#Parallel tempering prameters
T_max = 3.
n_chain = 4

#make sure this points to your white noise dictionary
noisefile = 'data/v1p1_all_dict.json'

#make sure this points to the RN empirical distribution file you plan to use (or set to None to not use empirical distributions)
rn_emp_dist_file = 'data/emp_dist_15yr_v1p1_bence_my_run_v4.3.pkl'
#rn_emp_dist_file = None

#file containing information about pulsar distances - None means use pulsar distances present in psr objects
#if not None psr objects must have zero distance and unit variance
psr_dist_file = 'data/pulsar_distances_15yr.pkl'

#this is where results will be saved
savefile = 'quickCW_test.h5'
#savefile = None

#Setup and start MCMC
#object containing common parameters for the mcmc chain
chain_params = ChainParams(T_max,n_chain, n_block_status_update,
                           freq_bounds=np.array([np.nan, 3e-7]), #prior bounds used on the GW frequency (a lower bound of np.nan is interpreted as 1/T_obs)
                           n_int_block=n_int_block, #number of iterations in a block (which has one shape update and the rest are projection updates)
                           save_every_n=save_every_n, #number of iterations between saving intermediate results (needs to be intiger multiple of n_int_block)
                           fisher_eig_downsample=fisher_eig_downsample, #multiplier for how much less to do more expensive updates to fisher eigendirections for red noise and common parameters compared to diagonal elements
                           rn_emp_dist_file=rn_emp_dist_file, #RN empirical distribution file to use (no empirical distribution jumps attempted if set to None)
                           savefile = savefile,#hdf5 file to save to, will not save at all if None
                           thin=100,  #thinning, i.e. save every `thin`th sample to file (increase to higher than one to keep file sizes small)
                           prior_draw_prob=0.2, de_prob=0.6, fisher_prob=0.3, #probability of different jump types
                           dist_jump_weight=0.2, rn_jump_weight=0.3, gwb_jump_weight=0.1, common_jump_weight=0.2, all_jump_weight=0.2, #probability of updating different groups of parameters
                           fix_rn=False, zero_rn=False, fix_gwb=False, zero_gwb=False) #switches to turn off GWB or RN jumps and keep them fixed and to set them to practically zero (gamma=0.0, log10_A=-20)

pta,mcc = QuickCW.QuickCW(chain_params, psrs,
                          amplitude_prior='detection', #specify amplitude prior to use - 'detection':uniform in log-amplitude, 'UL': uniform in amplitude
                          psr_distance_file=psr_dist_file, #file to specify advanced (parallax+DM) pulsar distance priors, if None use regular Gaussian priors based on pulsar distances in pulsar objects
                          noise_json=noisefile)

#Some parameters in chain_params can be updated later if needed
mcc.chain_params.thin = 10

67
Using 4 temperature chains with a geometric spacing of 1.442.
Temperature ladder is:
 [1.         1.44224957 2.08008382 3.        ]
Began Main Loop
Found lower frequency bound of nan - Setting it to 1/T.
Begin Loading Pulsar Timing Array from Enterprise at   17.098s
Finished Loading Pulsar Timing Array from Enterprise at   17.099s
List of parameters in the model with their priors:
[0_cos_gwtheta:Uniform(pmin=-1, pmax=1), 0_cos_inc:Uniform(pmin=-1, pmax=1), 0_gwphi:Uniform(pmin=0, pmax=6.283185307179586), 0_log10_fgw:Uniform(pmin=-8.704031455030655, pmax=-6.522878745280337), 0_log10_h:Uniform(pmin=-18, pmax=-11), 0_log10_mc:Uniform(pmin=7, pmax=10), 0_phase0:Uniform(pmin=0, pmax=6.283185307179586), 0_psi:Uniform(pmin=0, pmax=3.141592653589793), B1855+09_cw0_p_dist:PXDist(dist=1.1820330969267139, err=0.1243509995584841), B1855+09_cw0_p_phase:Uniform(pmin=0, pmax=6.283185307179586), B1855+09_red_noise_gamma:Uniform(pmin=0, pmax=7), B1855+09_red_noise_log10_A:Uniform(pmin=-20, pmax=-11)

uniform  [  0   1   2   3   4   5   6   7   9  10  11  13  14  15  17  18  19  21
  22  23  25  26  27  29  30  31  33  34  35  37  38  39  41  42  43  45
  46  47  49  50  51  53  54  55  57  58  59  61  62  63  65  66  67  69
  70  71  73  74  75  77  78  79  81  82  83  85  86  87  89  90  91  93
  94  95  97  98  99 101 102 103 105 106 107 109 110 111 113 114 115 117
 118 119 121 122 123 125 126 127 129 130 131 133 134 135 137 138 139 141
 142 143 145 146 147 149 150 151 153 154 155 157 158 159 161 162 163 165
 166 167 169 170 171 173 174 175 177 178 179 181 182 183 185 186 187 189
 190 191 193 194 195 197 198 199 201 202 203 205 206 207 209 210 211 213
 214 215 217 218 219 221 222 223 225 226 227 229 230 231 233 234 235 237
 238 239 241 242 243 245 246 247 249 250 251 253 254 255 257 258 259 261
 262 263 265 266 267 269 270 271 273 274 275 276 277]
normal  []
lin exp  []
dm dist  [ 16  28  32  40  44  48  68  76  84  96 100 104 132 140 160 164 168 172
 176 184 192 204 220 228 232 

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Calculating RN fisher Eigenvectors
Number of Pulsars with Fisher Eigenvectors in Full Default:  21 Diagonal Default:  15 No Default:  31
Finished Creating Shared Info Objects at  144.399s
Distance prior sigmas calculated for fisher correction:
[1.24351000e-01 2.01286315e-01 9.28380000e-01 1.08731563e-01
 3.58494739e-03 3.42320000e-01 3.44100000e-01 1.00799133e-03
 2.89360000e-01 5.74860000e-01 1.39520000e-01 1.40784780e-01
 1.03648306e-01 2.88120998e-01 1.88494730e-01 3.59480000e-01
 1.90988709e-01 3.75880000e-01 2.15517241e-02 4.99160000e-01
 1.94508283e-02 4.19836823e-02 1.25180000e-01 1.68560000e-01
 2.29140000e-01 2.24466891e-01 2.57759056e-01 2.63702649e-02
 2.40074192e-02 9.46850145e-02 5.93240271e-02 3.23760000e-01
 1.94139322e-02 2.41980000e-01 2.15787761e-02 1.02123085e-01
 5.64469706e-01 9.25890205e-03 2.54400000e-01 7.00160000e-01
 2.21280000e-01 5.91960000e-01 3.58140000e-01 4.66133601e-01
 3.43000000e-01 1.68172296e-01 1.29732000e+00 1.34269821e-02
 4.09145011e-01 4.150400

In [3]:
%%time
#Once we have everything set up, we can sample simply by asking mcc to advance the sampler
#With the above lightweight setup it should run in about an hour, but those should be increased for actual
#analysis, so these runs will typically take a few days
mcc.advance_N_blocks(N_blocks)

Entering Loop Body at  247.269s
Progress: 0.00% at t=   11.293s forecast loop time      nans
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     No Trials  No Trials  No Trials  No Trials 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    No Trials  No Trials  No Trials  No Trials 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      No Trials  No Trials  No Trials  No Trials 
GWB DE         No Trials  No Trials  No Trials  No Trials 
GWB Fisher     No Trials  No Trials  No Trials  No Trials 
Common Prior   No Trials  No Trials  No Trials  No Trials 
Common DE      No Trials  No Trials  No Trials  No Trials 
Common Fisher  No Trials  No Trials  No Trials  No Trials 
All DE         No Trials  No Trials  No Trials  No Trials 
All Fisher     No Trials  No Trials  No Trials  No Trials 
PT             No Trials  No Trials  No Tr

new best global sample old logL=+7974038.879 new logL=+7974039.385
local index of global best is (0, 23231)
new best params [  0.97127057   0.05329204   1.82718497  -7.54037877 -13.42602957
   9.61563404   6.26595483   1.76460964   1.31848811   3.36392636
   3.42593011 -14.65185059   3.09322443   2.08698236   3.95638958
 -13.66713151   5.46537741   5.56804402   3.38792884 -13.04727582
   0.91313569   5.71806614   3.07594899 -16.2715313    0.33531124
   1.59019473   5.89807204 -16.08549916   2.19489653   0.11937288
   4.647983   -17.64806163   2.07068284   4.28727438   1.96810709
 -15.70972813   0.15465785   6.07881322   0.74809881 -13.09504609
   1.33021836   3.12274546   0.04315762 -11.52056586   2.66367217
   2.80662395   5.40999337 -18.74037265   0.72926932   3.11483549
   2.25937578 -15.71992331   1.33697646   3.33030626   3.58757932
 -13.37728883   1.06981252   3.29899263   4.97683272 -13.76836187
   0.88558081   4.64977484   5.1601888  -15.16446559   1.5124303
   2.49214576   5.3

new best global sample old logL=+7974862.785 new logL=+7974874.523
local index of global best is (0, 59291)
new best params [-8.70504913e-01 -8.61249161e-01  5.53824592e+00 -8.27687594e+00
 -1.46445384e+01  9.24178102e+00  5.25733454e+00  1.99705182e+00
  1.21510181e+00  4.72384104e+00  3.94786410e+00 -1.44496998e+01
  2.92341773e+00  1.76119273e+00  4.03523773e+00 -1.35921561e+01
  3.88515635e+00  1.42849706e+00  3.28699446e-01 -1.26373849e+01
  8.79891286e-01  2.14466192e+00  9.98980938e-01 -1.34449188e+01
  3.26694984e-01  4.89717558e+00  2.62660416e+00 -1.62376698e+01
  2.13529830e+00  2.69438252e+00  6.71838807e+00 -1.70193547e+01
  2.00719678e+00  2.92405291e+00  1.40769324e+00 -1.55017627e+01
  1.54044603e-01  5.82950159e+00  2.30856358e-01 -1.33432326e+01
  1.78649151e+00  4.27267017e+00  3.02079636e-01 -1.22615034e+01
  2.86309826e+00  3.41031600e+00  9.16114424e-01 -1.50459393e+01
  7.36615107e-01  6.14392867e+00  3.38493457e+00 -1.52052923e+01
  1.41580643e+00  4.60909177e+0

new best global sample old logL=+7974878.560 new logL=+7974879.492
local index of global best is (0, 78399)
new best params [ -0.87050491  -0.70698339   5.53824592  -8.27687594 -14.42442487
   9.24178102   5.05303922   1.92690181   1.21510181   0.49353208
   3.9478641  -14.44969981   2.92341773   0.44884276   4.03523773
 -13.59215612   3.88515635   4.20689848   0.32869945 -12.63738492
   0.87989129   0.62246213   0.99898094 -13.44491879   0.32669498
   4.73068893   2.62660416 -16.23766977   2.1352983    1.29879571
   6.71838807 -17.0193547    2.00719678   0.36943471   1.40769324
 -15.50176275   0.1540446    0.94683007   0.23085636 -13.34323259
   1.78649151   0.9794593    0.30207964 -12.26150339   2.86309826
   0.19464433   0.91611442 -15.04593926   0.73661511   1.13713288
   3.38493457 -15.20529228   1.41580643   0.87896176   4.84055623
 -12.97196776   1.36476831   0.29416985   3.50539166 -13.78421752
   1.17769545   2.30019124   4.8980026  -17.34705707   1.22647347
   0.83508856   0.

mean likelihoods [7974874.18694937 7974773.8750308  7974592.65016798 7974516.11840422]
best likelihood anywhere in latest block 7974877.90858548
new best global sample old logL=+7974880.977 new logL=+7974881.188
local index of global best is (0, 11415)
new best params [ -0.87050491  -0.51954843   5.53824592  -8.27687594 -14.38839709
   9.24178102   5.12597405   1.79324675   1.07007503   3.84339095
   3.94674129 -14.45040524   2.66517996   2.18329175   3.98851874
 -13.60230253   5.95603753   2.89427638   0.44922376 -12.58848369
   1.04040395   4.94676706   0.88504008 -13.56004635   0.33114802
   3.16194564   2.5279484  -16.33632553   2.1094751    0.87611291
   6.71278467 -17.02495808   1.36524774   1.72340815   1.51691885
 -15.39253714   0.15541907   0.59130104   0.31961178 -13.34270451
   1.39094295   6.25217902   0.32533001 -12.25284623   3.1903107
   1.73993937   0.96695184 -14.99510184   0.56586393   2.34766614
   3.36461966 -15.2256072    1.19130721   0.28304845   4.86973604
 -12.9

...

mean likelihoods [7974918.79394124 7974885.68195136 7974869.31548554 7974868.74312157]
best likelihood anywhere in latest block 7974920.359752267
Progress: 71.00% at t= 2059.096s forecast loop time 2941.172s
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     0.9032258  1.0000000  1.0000000  1.0000000 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    0.0571429  0.1315789  0.1363636  0.1666667 
RN Emp Dist    0.2941176  0.1621622  0.0285714  0.0625000 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      0.2812500  0.4000000  0.3968254  0.4166667 
GWB DE         0.0526316  0.1250000  0.0588235  0.0384615 
GWB Fisher     0.4545455  0.2000000  0.0833333  0.2666667 
Common Prior   0.1666667  1.0000000  0.8000000  0.8983051 
Common DE      0.2708333  0.6750000  0.8333333  No Trials 
Common Fisher  0.3478261  0.8888889  0.928571

Progress: 78.00% at t= 2268.004s forecast loop time 2948.003s
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     0.9062500  1.0000000  1.0000000  1.0000000 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    0.0500000  0.1463415  0.1372549  0.1578947 
RN Emp Dist    0.2542373  0.1428571  0.0250000  0.0789474 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      0.2794118  0.4320988  0.3939394  0.4216867 
GWB DE         0.0500000  0.1200000  0.0540541  0.0357143 
GWB Fisher     0.4615385  0.1818182  0.0769231  0.2666667 
Common Prior   0.3750000  1.0000000  0.8181818  0.8939394 
Common DE      0.2941176  0.6744186  0.8529412  No Trials 
Common Fisher  0.4000000  0.8947368  0.9310345  No Trials 
All DE         0.0465116  0.0200000  0.0930233  0.0869565 
All Fisher     0.0645161  0.0967742  0.2162162  0.2083333 
PT            

mean likelihoods [7974931.92174567 7974883.40519926 7974876.01110237 7974874.91179072]
best likelihood anywhere in latest block 7974940.232729687
Rejected due to too fast evolution.
Progress: 85.00% at t= 2483.695s forecast loop time 2961.688s
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     0.8611111  1.0000000  1.0000000  1.0000000 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    0.0714286  0.1304348  0.1454545  0.1500000 
RN Emp Dist    0.2500000  0.1333333  0.0238095  0.0714286 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      0.2602740  0.4606742  0.3802817  0.4157303 
GWB DE         0.0416667  0.1071429  0.0512821  0.0344828 
GWB Fisher     0.4285714  0.1538462  0.0769231  0.2105263 
Common Prior   0.2727273  1.0000000  0.8461538  0.9027778 
Common DE      0.2678571  0.6956522  0.8684211  No Trials 
Common Fi

Rejected due to too fast evolution.
Progress: 92.00% at t= 2692.386s forecast loop time 2965.677s
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     0.7948718  1.0000000  1.0000000  1.0000000 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    0.0869565  0.1276596  0.1379310  0.1587302 
RN Emp Dist    0.2753623  0.1224490  0.0217391  0.0588235 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      0.2500000  0.4838710  0.4000000  0.4183673 
GWB DE         0.0400000  0.1034483  0.0465116  0.0625000 
GWB Fisher     0.4285714  0.1333333  0.1333333  0.2000000 
Common Prior   0.2500000  1.0000000  0.8571429  0.8987342 
Common DE      0.2459016  0.7083333  0.8604651  No Trials 
Common Fisher  0.3703704  0.9166667  0.9393939  No Trials 
All DE         0.0566038  0.0158730  0.0892857  0.0740741 
All Fisher     0.0588235  0.1081081  

mean likelihoods [7974913.23077129 7974883.80938989 7974863.92874875 7974862.73493774]
best likelihood anywhere in latest block 7974917.659931746
Rejected due to too fast evolution.
Progress: 99.00% at t= 2905.278s forecast loop time 2973.344s
Acceptance fractions for various jumps
Jump Name      chain   0  chain   1  chain   2  chain   3 
Temperature    1.000e+00  1.442e+00  2.080e+00  3.000e+00 
Dist Prior     0.7948718  1.0000000  1.0000000  1.0000000 
Dist DE        No Trials  No Trials  No Trials  No Trials 
Dist Fisher    0.0800000  0.1372549  0.1269841  0.1515152 
RN Emp Dist    0.2800000  0.1250000  0.0181818  0.0517241 
RN DE          No Trials  No Trials  No Trials  No Trials 
RN Fisher      0.2808989  0.4700000  0.4102564  0.4326923 
GWB DE         0.0384615  0.1000000  0.0425532  0.0588235 
GWB Fisher     0.4666667  0.1250000  0.1176471  0.1904762 
Common Prior   0.2000000  1.0000000  0.8666667  0.9012346 
Common DE      0.2812500  0.7083333  0.8723404  No Trials 
Common Fi

Once the run is done, results can be analyzed in the `detection_analysis.ipynb` and `UL_analysis.ipynb` notebooks.